## Imports

In [36]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [37]:
df = pd.read_csv('redfin_2023-08-03-08-12-35.csv')

## Data Clean

In [38]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [39]:
df = df.dropna(subset=['SOLD DATE'])

In [40]:
# Define list of desired months (excluding current month)
desired_months = ['July']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [41]:
df_filtered.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [42]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    859
Name: count, dtype: int64
-------
$/SQUARE FEET
False    844
True      15
Name: count, dtype: int64
-------
YEAR BUILT
False    859
Name: count, dtype: int64
-------


In [43]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [44]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [45]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
727,PAST SALE,July-18-2023,Condo/Co-op,1100 SE 5th Ct #33,Pompano Beach,FL,33060.0,625.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1100-S...,Beaches MLS,F10369221,N,Y,26.222451,-80.107146
198,PAST SALE,July-27-2023,Condo/Co-op,1428 SE 4 Th Ave #220,Deerfield Beach,FL,33441.0,54000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1428...,MARMLS,A11411214,N,Y,26.299167,-80.096518
246,PAST SALE,July-14-2023,Condo/Co-op,8851 Sunrise Lakes Blvd #112,Sunrise,FL,33322.0,75000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/8851-Sunrise...,Beaches MLS,F10384679,N,Y,26.163868,-80.269112
622,PAST SALE,July-26-2023,Condo/Co-op,7310 NW 1st St #204,Margate,FL,33063.0,77500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/7310-NW-1st-...,Beaches MLS,F10378978,N,Y,26.230777,-80.223808
772,PAST SALE,July-14-2023,Condo/Co-op,8120 SW 24th St #109,North Lauderdale,FL,33068.0,78000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Lauderdale/812...,Beaches MLS,F10366840,N,Y,26.198492,-80.236448
581,PAST SALE,July-3-2023,Condo/Co-op,286 Oakridge Q #286,Deerfield Beach,FL,33442.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/286-...,Beaches MLS,F10372433,N,Y,26.305770,-80.139976
150,PAST SALE,July-14-2023,Condo/Co-op,2751 E Sunrise Lakes Dr #306,Sunrise,FL,33322.0,80500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/2751-E-Sunri...,MARMLS,A11398442,N,Y,26.159563,-80.254296
649,PAST SALE,July-17-2023,Condo/Co-op,2651 E Sunrise Lakes Dr #305,Sunrise,FL,33322.0,83000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/2651-E-Sunri...,Beaches MLS,F10371433,N,Y,26.158139,-80.255134
316,PAST SALE,July-21-2023,Condo/Co-op,3001 NW 48th Ave #143,Lauderdale Lakes,FL,33313.0,85000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/300...,Beaches MLS,F10384726,N,Y,26.163051,-80.213450
684,PAST SALE,July-1-2023,Condo/Co-op,7355 NW 5th Pl #107,Margate,FL,33063.0,85000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Margate/7355-NW-5th-...,Beaches MLS,F10338166,N,Y,26.237628,-80.223984


In [46]:
print(df_filtered['URL'].iloc[727])

https://www.redfin.com/FL/Pompano-Beach/1100-SE-5th-Ct-33060/unit-33/home/41631436


In [47]:
# Correct the prices, if needed
df_filtered.at[727,'PRICE']=(625000)

In [48]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [49]:
# # Corrections, if needed
df_filtered.at[727,'$/SQUARE FEET']=(625000/1170)

In [50]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
684,85000.0,7355 NW 5th Pl #107,Margate,86.0
772,78000.0,8120 SW 24th St #109,North Lauderdale,92.0
519,103000.0,4401 NW 41st St #307,Lauderdale Lakes,93.0
781,90000.0,4740 NW 21st St #408,Lauderhill,94.0
651,87000.0,4851 NW 21st St #102,Lauderhill,94.0
846,90000.0,5001 NW 34th St #203,Lauderdale Lakes,95.0
770,90500.0,4770 NW 21st St #203,Lauderhill,98.0
470,93000.0,3776 Inverrary Blvd Unit 207R,Lauderhill,103.0
817,90000.0,9441 Sunrise Lakes Blvd #109,Sunrise,105.0
570,91000.0,6001 N Falls Cir Dr #406,Lauderhill,105.0


In [51]:
# print(df_filtered.URL.iloc[953])

In [52]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [53]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [54]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [55]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [56]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [57]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [58]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"June 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [59]:
m.save('index.html')

## Summary Info

In [60]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [61]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [62]:
# # Find the second lowest value
# second_lowest_value = df_filtered['YEAR BUILT'].nsmallest(2).iloc[-1]

# # Get the row associated with the second lowest value
# row_with_second_lowest = df_filtered[df_filtered['YEAR BUILT'] == second_lowest_value]

# # Print the row
# print(row_with_second_lowest)

In [77]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Fort-Lauderdale/741-Bayshore-Dr-33304/unit-32N/home/41782284


In [64]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | psf: ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | psf: ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | psf: ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | psf: ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | psf: ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f}  | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | psf: ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f}  | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
RIVA, 1180 N Federal Hwy #1602 Fort Lauderdale | Price $4,625,000 | psf: $1,107 | Year built: 2018
Least Expensive
DEERFIELD PINES CONDO, 1428 SE 4 Th Ave #220 Deerfield Beach | Price $54,000 | psf: $nan | Year built: 1973
Highest Price Per Square Foot
Villas By The Sea, 4510 El Mar Dr #402 Lauderdale By The Sea | Price $2,995,000 | psf: $1,506 | Year built: 2016
Lowest Price Per Square Foot
Oriole Gardens, 7355 NW 5th Pl #107 Margate | Price $85,000 | psf: $86 | Year built: 1973
Newest
METROPICA NORTH TOWER ONE, 2000 Metropica Way #1706 Sunrise | Price $611,000 | psf: $582  | Year built: 2020
Oldest
Holiday Isle Yacht Club Condo, 741 Bayshore Dr Unit 32n Fort Lauderdale | Price $365,500 | psf: $649  | Year built: 1951


## Time on Market Calculator

In [65]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [74]:
from datetime import datetime, timedelta

date1 = datetime(2023, 6, 28) ## List (Earlier) date
date2 = datetime(2023, 7, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

29


## Map URL Snagger

In [67]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [68]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_condo_sales_month_ending_july_2023


## Get Summary Data

In [69]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 859
--------
Total sale price: $268,809,858
--------
Median sale price: $241,000
--------
Max sale price: $4,625,000
--------
Min sale price: $54,000
------------------------------------------------
PSF INFO
Max price per square foot: $1,506
--------
Min price per square foot: $86
--------
Median price per square foot: $234
------------------------------------------------
CONDO AGES
Newest building: 2020.0
----------
Oldest building: 1951.0
----------
Average building age: 1980.5378346915018
